In [33]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install tqdm
!pip install yahoo_fin
!pip install "git+https://github.com/tqdm/tqdm.git@devel#egg=tqdm"

In [2]:
import datetime
import pandas as pd # pip install pandas
import requests # pip install requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import yfinance as yf
from tqdm import tqdm
import numpy as np
from yahoo_fin import options
from tqdm import tqdm

In [3]:
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
}
url = 'https://www.marketwatch.com/tools/earningscalendar'
r = requests.get(url, headers = headers)
# Request with url and header
soup = BeautifulSoup(r.content, 'html.parser')
tabpane = soup.find('div', 'tabpane')
earning_tables = tabpane.find_all('div', 'daypage')
current_datetime = datetime.date.today()
tomorrow = current_datetime.strftime('%b_%d')
anticipated_earnings = pd.DataFrame()
print('Preparing to Scrap Data...')
for earning_table in tqdm(earning_tables):
    earning_date = earning_table['id'].replace('page', '')
    earning_date = earning_date[:3] + '_' + earning_date[3:]
    if tomorrow == earning_date:
        table = earning_table.find("table")
        pretty = table.prettify()
        pd_table = pd.read_html(pretty)
        pd_table_1 = pd_table[0]
        anticipated_earnings = pd_table_1
print('Finished Scraping Data...')
anticipated_earnings.columns = ["Company Name", "Symbol", "Fiscal Quarter", "EPS Forecast", "EPS Actual", "Suprise"]
pd.set_option("display.max_rows", None, "display.max_columns", None)
anticipated_earnings = anticipated_earnings.drop(columns=["Suprise", "EPS Actual", "Fiscal Quarter","EPS Forecast"])
eligible_tickers = anticipated_earnings
print('Finished Processing Data...')

100%|██████████| 17/17 [00:00<00:00, 234.02it/s]

Preparing to Scrap Data...
Finished Scraping Data...
Finished Processing Data...


In [4]:
print(eligible_tickers)

                                          Company Name Symbol
0                             Envirotech Vehicles Inc.   EVTV
1                                         CureVac N.V.   CVAC
2                                   Biofrontera AG ADR  BFAGY
3                             Cemex S.A.B. de C.V. ADR     CX
4                 PTT Exploration & Production PCL ADR  PEXNY
5                                   Axis Bank Ltd. GDR  AXBKY
6                             Barfresh Food Group Inc.   BRFH
7                                           Zynex Inc.   ZYXI
8                                    Daibiru Corp. ADR  DIBUY
9               LG Electronics Inc. New Sp GDR 144A Nv  LGEJY
10                               RiceBran Technologies   RIBT
11                                  Carlyle Group Inc.     CG
12                                     Clariant AG ADR  CLZNY
13                             Ambuja Cements Ltd. GDR  AMBUY
14                                     VK Co. Ltd. GDR  MLRYY
15      

In [5]:
current_week = datetime.date.today().isocalendar()[1]
today_formatted = datetime.date.today().strftime("%Y-%m-%d")

In [6]:
raw_data = []
for index, row in tqdm(eligible_tickers.iterrows()):
    try:
        dates = options.get_expiration_dates(row['Symbol'])
        if len(dates) != 0:
            closest_options_week = datetime.datetime.strptime(dates[0], '%B %d, %Y').isocalendar()[1]
            if closest_options_week - current_week < 2:
                current_ticker = float(yf.download(row['Symbol'],today_formatted, progress=False)['Adj Close'])
                chain = options.get_options_chain(row['Symbol'],dates[0])["puts"]
                total_open_interest = chain["Open Interest"].replace('-','0').astype('int').sum()
                total = chain["Implied Volatility"].str.rstrip('%').astype('float') / 100.0
                total[chain["Implied Volatility"].str.rstrip('%').astype('float') / 100.0 == 0.0] = np.nan
                average_iv = np.nanmean(total)
                temp = [row['Symbol'], current_ticker , dates[0], total_open_interest, average_iv]
                raw_data.append(temp)
    except Exception as e:
        print(row['Symbol'])
        print(e)
# final_stock_options.columns = ["Ticker", "Price", "Clos Exp", "Tot OI", "Avg IV"]
# print(final_stock_options)


67it [00:44,  1.12it/s]

could not convert string to float: '1,055.47'


107it [01:18,  1.60s/it]

No tables found


262it [4:04:32, 4284.89s/it]

HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out. (read timeout=None)


379it [4:07:50, 39.24s/it]  


In [7]:
raw_data

[['NOK', 4.900000095367432, 'April 29, 2022', 50547, 1.2254571428571428],
 ['LLY', 297.2699890136719, 'April 29, 2022', 2663, 1.3151454545454548],
 ['MRK', 88.58000183105469, 'April 29, 2022', 10771, 1.0082185185185186],
 ['CAT', 212.44000244140625, 'April 29, 2022', 12800, 1.6189399999999996],
 ['SO', 75.58000183105469, 'April 29, 2022', 2558, 1.094878947368421],
 ['NOC', 445.1700134277344, 'April 29, 2022', 1993, 1.3362085714285714],
 ['ROKU', 91.62999725341797, 'April 29, 2022', 0, 0.39915],
 ['FSLR', 72.26000213623047, 'April 29, 2022', 0, 0.3687533333333333]]

In [8]:
final_stock_options = pd.DataFrame(raw_data)
final_stock_options.columns = ["Ticker", "Price", "Clos Exp", "Tot OI", "Avg IV"]
print(final_stock_options)

  Ticker       Price        Clos Exp  Tot OI    Avg IV
0    NOK    4.900000  April 29, 2022   50547  1.225457
1    LLY  297.269989  April 29, 2022    2663  1.315145
2    MRK   88.580002  April 29, 2022   10771  1.008219
3    CAT  212.440002  April 29, 2022   12800  1.618940
4     SO   75.580002  April 29, 2022    2558  1.094879
5    NOC  445.170013  April 29, 2022    1993  1.336209
6   ROKU   91.629997  April 29, 2022       0  0.399150
7   FSLR   72.260002  April 29, 2022       0  0.368753


In [9]:
print('Sorted by stock price')
final_stock_options.sort_values(by=['Price'])

Sorted by stock price


,Ticker,Price,Clos Exp,Tot OI,Avg IV
0,NOK,4.900000,"April 29, 2022",50547,1.225457
7,FSLR,72.260002,"April 29, 2022",0,0.368753
4,SO,75.580002,"April 29, 2022",2558,1.094879
2,MRK,88.580002,"April 29, 2022",10771,1.008219
6,ROKU,91.629997,"April 29, 2022",0,0.399150
3,CAT,212.440002,"April 29, 2022",12800,1.618940
1,LLY,297.269989,"April 29, 2022",2663,1.315145
5,NOC,445.170013,"April 29, 2022",1993,1.336209


In [10]:
print('Sorted by Open Interest')
final_stock_options.sort_values(by=['Tot OI'], ascending=False)

Sorted by Open Interest


,Ticker,Price,Clos Exp,Tot OI,Avg IV
0,NOK,4.900000,"April 29, 2022",50547,1.225457
3,CAT,212.440002,"April 29, 2022",12800,1.618940
2,MRK,88.580002,"April 29, 2022",10771,1.008219
1,LLY,297.269989,"April 29, 2022",2663,1.315145
4,SO,75.580002,"April 29, 2022",2558,1.094879
5,NOC,445.170013,"April 29, 2022",1993,1.336209
6,ROKU,91.629997,"April 29, 2022",0,0.399150
7,FSLR,72.260002,"April 29, 2022",0,0.368753


In [11]:
print('Sort by Average IV')
final_stock_options.sort_values(by=['Avg IV'], ascending=False)

Sort by Average IV


,Ticker,Price,Clos Exp,Tot OI,Avg IV
3,CAT,212.440002,"April 29, 2022",12800,1.618940
5,NOC,445.170013,"April 29, 2022",1993,1.336209
1,LLY,297.269989,"April 29, 2022",2663,1.315145
0,NOK,4.900000,"April 29, 2022",50547,1.225457
4,SO,75.580002,"April 29, 2022",2558,1.094879
2,MRK,88.580002,"April 29, 2022",10771,1.008219
6,ROKU,91.629997,"April 29, 2022",0,0.399150
7,FSLR,72.260002,"April 29, 2022",0,0.368753


In [1]:
filtered_stocks = final_stock_options
cash = 2932
eligible_price = float(cash / 100)
for index, row in filtered_stocks.iterrows():
    try:
        if (row['Price'] > eligible_price):
            filtered_stocks.drop(index, inplace=True)
    except Exception as e::
        print(e)

NameError: name 'final_stock_options' is not defined

In [ ]:
filtered_stocks